<a href="https://colab.research.google.com/github/subha-v/FatimaFellowshipProjectPublic/blob/main/RealFatima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here we will import the necessary packages for the project.

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Here we will import the datasets into the project.

In [13]:
fakenews = pd.read_csv('/content/Fake.csv')
truenews = pd.read_csv('/content/True.csv')

We're looking at the datasets to get a general idea of how they are formatted

In [ ]:
fakenews.head()

We are combining both the datasets into one dataset and also adding a column regarding whether the news is true or fake

In [15]:
fakenews['temp']= 0
truenews['temp']= 1

combined_dataset = pd.DataFrame()
combined_dataset = truenews.append(fakenews)

Taking a look at the new dataset

In [ ]:
combined_dataset

We don't need the columns of date and subject for right now

In [17]:
column = ['date','subject']
combined_dataset = combined_dataset.drop(columns=column)

We can combine the title of the article with the body text and drop the title as well

In [63]:
combined_dataset["text"] = combined_dataset["text"]+" "+ combined_dataset["title"]

In [64]:
combined_dataset = combined_dataset.drop(columns="title")

In [62]:
# now we will process the data using NLTK
import nltk
import re
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [66]:
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.stem.snowball import SnowballStemmer
st = SnowballStemmer('english')
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [67]:
def remove_punctuation(text):  
    return re.sub(r'[^\w\s]','', str(text))
                  
                  
def remove_stopwords(text):  #Removing stopwords(eg. this, that, am, be etc)
    stop = stopwords.words("english")
    final_text = []
    for i in str(text).split():
        if i.strip() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
                  
def tokenize(text):  
    tokens = re.split('\W+',text) #W+ means that either a word character (A-Z) or a dash(-) can go there.
    return tokens

                          
def stemming(text):  
    porter_stemmer = PorterStemmer()
    return porter_stemmer.stem(str(text))
                  
def lemmatization(text):  #Applying Lemaatization 
    wordnet_lemmatizer = WordNetLemmatizer()
    return wordnet_lemmatizer.lemmatize(str(text))

In [69]:
  >>> import nltk
  >>> nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [70]:
def clean_text(text):
    text = remove_punctuation(text)
    text = remove_stopwords(text)
    text = stemming(text)
    return lemmatization(text)

combined_dataset['text'] = combined_dataset['text'].apply(lambda x: clean_text(x))

In [72]:
all_text = combined_dataset['text']

In [73]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
all_text = cv.fit_transform(all_text)

In [74]:
truth_or_false = combined_dataset['temp']

In [75]:
from keras.preprocessing import sequence, text
from sklearn.model_selection import train_test_split
xtrain, xvalid, ytrain, yvalid = train_test_split(all_text, truth_or_false, 
                                                  random_state=42, 
                                                  test_size=0.15, shuffle=True)


In [82]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(xtrain,ytrain)
predictions2 = LR.predict(xvalid)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [83]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(yvalid,predictions2))
print('\n')
print(classification_report(yvalid,predictions2))

[[3529    0]
 [   0 3206]]


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3529
           1       1.00      1.00      1.00      3206

    accuracy                           1.00      6735
   macro avg       1.00      1.00      1.00      6735
weighted avg       1.00      1.00      1.00      6735



In [27]:
# A Naive Bayes Classifier determines the probability that an example belongs to some class, calculating the probability that an event will occur given that some input event has occurred.

In [76]:
from sklearn.naive_bayes import MultinomialNB

In [77]:
nb = MultinomialNB()

In [78]:
nb.fit(xtrain, ytrain)

MultinomialNB()

In [79]:
predictions = nb.predict(xvalid)

In [80]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(yvalid,predictions))
print('\n')
print(classification_report(yvalid,predictions))

[[3393  133]
 [ 105 3104]]


              precision    recall  f1-score   support

           0       0.97      0.96      0.97      3526
           1       0.96      0.97      0.96      3209

    accuracy                           0.96      6735
   macro avg       0.96      0.96      0.96      6735
weighted avg       0.96      0.96      0.96      6735

